In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext()

In [7]:
data = sc.textFile('../data/Uber-Jan-Feb-FOIL.csv')

data.count()

355

In [14]:
# 打印第一行表头
data.first()

'dispatching_base_number,date,active_vehicles,trips'

In [13]:
# 第一个字段去重后计数
rows = data.map(lambda line : line.split(','))
rows.map(lambda row: row[0]).distinct().count()

7

In [16]:
# 第一个字段去重
rows.map(lambda row: row[0]).distinct().collect()

['dispatching_base_number',
 'B02765',
 'B02682',
 'B02598',
 'B02512',
 'B02764',
 'B02617']

# filter

In [19]:
# 过滤出dispatching_base_number == 'B02682'的数据
# data_2682 = rows.filter(lambda row: row[0] == 'B02765')
data_2682 = rows.filter(lambda row: 'B02682' in row)
data_2682.collect()

[['B02682', '1/1/2015', '945', '7679'],
 ['B02682', '1/2/2015', '890', '5506'],
 ['B02682', '1/3/2015', '915', '8010'],
 ['B02682', '1/4/2015', '812', '5621'],
 ['B02682', '1/5/2015', '951', '6012'],
 ['B02682', '1/6/2015', '1022', '7491'],
 ['B02682', '1/7/2015', '1039', '9078'],
 ['B02682', '1/8/2015', '1135', '10416'],
 ['B02682', '1/9/2015', '1140', '10477'],
 ['B02682', '1/10/2015', '1057', '11629'],
 ['B02682', '1/11/2015', '943', '8461'],
 ['B02682', '1/12/2015', '1082', '9107'],
 ['B02682', '1/13/2015', '1092', '9629'],
 ['B02682', '1/14/2015', '1174', '9762'],
 ['B02682', '1/15/2015', '1208', '10391'],
 ['B02682', '1/16/2015', '1250', '11280'],
 ['B02682', '1/17/2015', '1137', '11382'],
 ['B02682', '1/18/2015', '1056', '11161'],
 ['B02682', '1/19/2015', '883', '7028'],
 ['B02682', '1/20/2015', '1151', '8496'],
 ['B02682', '1/21/2015', '1228', '9472'],
 ['B02682', '1/22/2015', '1295', '10699'],
 ['B02682', '1/23/2015', '1330', '11767'],
 ['B02682', '1/24/2015', '1223', '13355']

In [20]:
data_2617 =rows.filter(lambda row: 'B02617' in row)
data_2617.filter(lambda row: int(row[3]) > 15000).count()

6

In [23]:
# 过滤表头
noheader_data = sc.textFile('../data/Uber-Jan-Feb-FOIL.csv').filter(lambda line : 'base' not in line)
# 每行逗号分隔
noheader_data = noheader_data.map(lambda line: line.split(','))
# 计算每个dispatching_base_number，的trip和
trip_sum = noheader_data.map(lambda line: (line[0], int(line[3]))).reduceByKey(lambda x,y: x+y)
# 从大到小，取前三, 根据value排序
sorted_trips = trip_sum.takeOrdered(10, key=lambda x: -x[1])
#
sorted_trips

[('B02764', 1914449),
 ('B02617', 725025),
 ('B02682', 662509),
 ('B02598', 540791),
 ('B02765', 193670),
 ('B02512', 93786)]